In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter

pd.options.display.max_columns = None

In [122]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
url2 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
df_usconf = pd.read_csv(url,error_bad_lines = False)
df_usdead = pd.read_csv(url2,error_bad_lines = False)

In [181]:
us_conf_daily = df_usconf.copy()
us_dead_daily = df_usdead.copy()

us_conf_daily.iloc[:,11:] = us_conf_daily.iloc[:,11:].diff(axis=1).fillna(0).astype('int')
us_dead_daily.iloc[:,12:] = us_dead_daily.iloc[:,12:].diff(axis=1).fillna(0).astype('int')

us_conf_daily = us_conf_daily.groupby(['Province_State'])[us_conf_daily.columns.to_list()[11:]].agg('sum').reset_index()
us_dead_daily = us_dead_daily.groupby(['Province_State'])[us_dead_daily.columns.to_list()[11:]].agg('sum').reset_index()

us_conf_daily = us_conf_daily.melt(id_vars = us_conf_daily.columns[:1], var_name = 'Date', value_name = 'Daily Confirmed Cases').sort_values(by = ['Province_State','Date']).reset_index(drop=True)
us_dead_daily = us_dead_daily.melt(id_vars = us_dead_daily.columns[:2], var_name = 'Date', value_name = 'Daily Dead').sort_values(by = ['Province_State','Date']).reset_index(drop=True)

us_daily = us_conf_daily.merge(us_dead_daily[['Province_State','Population','Date','Daily Dead']], on = ['Province_State','Date'], how = 'left', suffixes = (False, False))
us_daily = us_daily[['Province_State','Population','Date', 'Daily Confirmed Cases','Daily Dead']]
ca_daily = us_daily.loc[us_daily['Province_State']=='California'].copy().reset_index(drop=True)
ca_daily['Date'] = pd.to_datetime(ca_daily['Date'])

In [182]:
url = 'https://covidtracking.com/api/v1/states/daily.csv'
df_testing = pd.read_csv(url,error_bad_lines = False)
df_testing = df_testing.rename(columns = {'date':'Date','state':'Province_State'})

ca_testing = df_testing[['Date','Province_State','positive','negative','hospitalizedCurrently','inIcuCurrently', 'death', 'positiveIncrease','negativeIncrease','totalTestResults','totalTestResultsIncrease','deathIncrease']].loc[df_testing['Province_State'] =='CA'].copy().reset_index(drop=True)
ca_testing['Date'] = pd.to_datetime(ca_testing['Date'].astype(dtype = 'str'))
ca_testing['Province_State'] = 'California'

In [183]:
ca_daily = ca_daily.merge(ca_testing[['Province_State', 'Date','totalTestResults','positive','negative','hospitalizedCurrently','inIcuCurrently']], on = ['Province_State', 'Date'], how = 'left', suffixes = (False, False))
ca_daily.tail()

,Province_State,Population,Date,Daily Confirmed Cases,Daily Dead,totalTestResults,positive,negative,hospitalizedCurrently,inIcuCurrently
163,California,39512223,2020-07-03,3964,50,4448176.0,248235.0,4199941.0,7024.0,1871.0
164,California,39512223,2020-07-04,2381,19,4553031.0,254745.0,4298286.0,7092.0,1888.0
165,California,39512223,2020-07-05,11786,39,4680138.0,260155.0,4419983.0,7149.0,1907.0
166,California,39512223,2020-07-06,6354,68,4793353.0,271684.0,4521669.0,7278.0,1937.0
167,California,39512223,2020-07-07,12977,132,4896370.0,277774.0,4618596.0,7499.0,1984.0


In [184]:
ca_daily.to_csv(r'X:\AC\Documents\Datasets\CA_daily_pivot.csv')